<a id="top"></a>
# 03B — Búsqueda de hiperparámetros con Optuna (Continual Learning)

**Qué hace este notebook**  
Automatiza la **optimización de hiperparámetros (HPO)** para métodos de aprendizaje continuo manteniendo **coherencia total** con `configs/presets.yaml`. Permite afinar, entre otros:

- **naive** (línea base),
- **ewc** (Elastic Weight Consolidation),
- **rehearsal** (rejuego con buffer),
- **rehearsal+ewc** (combinación),
- **as-snn** (bio-inspirado; espacio de búsqueda incluido).

Se apoya en:
- `configs/presets.yaml` (misma configuración que el resto de cuadernos),
- `build_make_loader_fn` (elige **H5 offline** o **CSV + codificación en runtime**),
- `run_continual` (entrena, evalúa y guarda métricas en `outputs/`).

---

### Métrica objetivo (minimizar)

\[
\textbf{Objetivo}= \mathrm{MAE}_{\text{tarea final}} + \alpha \cdot \max\!\bigl(0,\; \text{OlvidoRelativo}\,\%\bigr)
\]

- **MAE de la tarea final**: rendimiento en la **última** tarea de la secuencia.  
- **Olvido relativo (%)**: degradación de la **primera** tarea tras aprender la/s siguiente/s.  
- **α**: peso del olvido (por defecto **0.5**). Súbelo si quieres penalizar más el olvido.

> La extracción de métricas se realiza desde `continual_results.json`. Si falta información, ese *trial* se considera peor (valor infinito) para no sesgar el estudio.

---

## ✅ Prerrequisitos
- `pip install optuna`  
- **Datos** preparados (`tasks.json` o `tasks_balanced.json` desde 01/01A).  
- Si el preset usa **offline** (`use_offline_spikes: true`), tener los **H5** generados con **02_ENCODE_OFFLINE** (mismo `encoder/T/gain/size/to_gray`).

> **Consejo**: empieza con el preset `fast` y pocos *trials*; si todo es estable, sube `N_TRIALS` y/o `epochs`.


<a id="toc"></a>

## 🧭 Índice
- [1) Imports y setup de entorno](#sec-01)  
- [2) Carga de preset y construcción de modelo/transform](#sec-02)  
- [3) Tareas y factory de loaders](#sec-03)  
- [4) Métricas y objetivo para Optuna](#sec-04)  
- [5) Espacios de búsqueda (por método)](#sec-05)  
- [6) Estudio Optuna — un método concreto](#sec-06)  
- [7) Estudio Optuna conjunto (elige método + HPs)](#sec-07)  
- [8) Re-entrena con los mejores hiperparámetros](#sec-08)  
- [9) Resumen rápido de *runs* (tabla)](#sec-09)



<a id="sec-01"></a>
## 1) Imports y setup de entorno

**Objetivo**  
Configurar el entorno de HPO de forma reproducible y eficiente:

- Limitar hilos BLAS (`OMP/MKL/OPENBLAS`) para evitar sobrecarga de CPU.  
- Detectar `ROOT` (raíz del repo) y añadirlo a `sys.path`.  
- Importar utilidades del proyecto (`load_preset`, `build_make_loader_fn`, `run_continual`, etc.).  
- Seleccionar dispositivo (`cuda` si está disponible) y activar optimizaciones de PyTorch (TF32/cuDNN).

> Si tu CPU va justa, baja `torch.set_num_threads(4)` a `2`.  

[↑ Volver al índice](#toc)

In [1]:
# Limitar threads BLAS (opcional)
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# Para fragmentación de memoria (PyTorch 2.x):
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from pathlib import Path
import sys, json, copy, time
import torch
import optuna

# Raíz del repo y sys.path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Proyecto
from src.utils import load_preset, build_make_loader_fn
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
from src.runner import run_continual

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("Device:", device)

/home/cesar/proyectos/TFM_SNN/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


<a id="sec-02"></a>
## 2) Carga de preset y construcción de modelo/transform

**Objetivo**  
Tomar la **fuente de la verdad** del experimento desde `configs/presets.yaml` y derivar:

- **Modelo/transform** (`ImageTransform`) según `img_w`, `img_h`, `to_gray`.  
- **Codificación temporal** (`encoder` ∈ `{rate, latency, raw}`, `T`, `gain`, `seed`).  
- **Flags de datos**: `use_offline_spikes` (H5 offline) y/o `encode_runtime` (codificación en GPU).  
- **DataLoader**: `num_workers`, `prefetch_factor`, `pin_memory`, `persistent_workers`.  
- **Augment** (`aug_train`) y **balanceo online** si procede.

Se define `make_model_fn(tfm)` para instanciar el modelo con los parámetros adecuados.  

[↑ Volver al índice](#toc)

In [2]:
PRESET = "std"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# Modelo / tfm
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"],
    CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=None
)

# Datos / codificación temporal
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

# Flags & loader
USE_OFFLINE_SPIKES = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE     = bool(CFG["data"].get("encode_runtime", False))

NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)      # robusto
PREFETCH    = int(CFG["data"].get("prefetch_factor") or 2)  # <- casteo robusto
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BAL = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BAL} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

def make_model_fn(tfm):
    # kwargs específicos de pilotnet_snn; ignorados para otros
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)


[PRESET=std] model=pilotnet_snn 200x66 gray=True
[DATA] encoder=rate T=16 gain=0.5 seed=42
[LOADER] workers=8 prefetch=2 pin=True persistent=True
[BALANCE] online=False bins=50
[RUNTIME_ENCODE] False | [OFFLINE_SPIKES] True


<a id="sec-03"></a>
## 3) Tareas y factory de loaders

**Objetivo**  
Construir la lista de tareas y un **factory de DataLoaders** coherente con el preset:

- Se elige `tasks_balanced.json` si `prep.use_balanced_tasks: true` y existe; si no, `tasks.json`.  
- Si `use_offline_spikes: true`, se verifican los **H5** esperados (nomenclatura fija con `encoder/T/gain/size/color`).  
- `build_make_loader_fn(...)` selecciona automáticamente **H5** (offline) o **CSV + runtime encode** en GPU.  
- El *wrapper* `make_loader_fn(...)` solo **propaga kwargs** (augment, balanceo online, *workers*, etc.) para que el *runner* no cambie.

> Si usas *tasks* balanceadas, el **train** debe ser `train_balanced.csv` (o su H5 derivado). El notebook lo comprueba.  

[↑ Volver al índice](#toc)

In [3]:
# Leer tasks.json / tasks_balanced.json (elige según el preset)
from pathlib import Path as _P
import json

PROC = ROOT / "data" / "processed"

USE_BALANCED = bool(CFG.get("prep", {}).get("use_balanced_tasks", False))
tb_name = (CFG.get("prep", {}).get("tasks_balanced_file_name") or "tasks_balanced.json")
t_name  = (CFG.get("prep", {}).get("tasks_file_name")           or "tasks.json")

cand_bal = PROC / tb_name
cand_std = PROC / t_name
TASKS_FILE = cand_bal if (USE_BALANCED and cand_bal.exists()) else cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)
task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

print("Usando:", TASKS_FILE.name)
print("Tareas y TRAIN CSV/H5 a usar:")
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# Guardarraíl: si balanced, exigir train_balanced.csv
if USE_BALANCED:
    for t in task_list:
        train_path = _P(tasks_json["splits"][t["name"]]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en modo balanced, pero encontré '{train_path.name}'."
            )

# Si usas H5 offline, chequear que existan
if USE_OFFLINE_SPIKES:
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []
    for t in task_list:
        base = PROC / t["name"]
        for split in ("train", "val", "test"):
            p = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not p.exists():
                missing.append(str(p))
    if missing:
        print("[WARN] Faltan H5. Genera primero con 02_ENCODE_OFFLINE.ipynb (o tools/encode_tasks.py).")

# Factory de loaders con kwargs del preset
from src.utils import build_make_loader_fn

_raw_make_loader_fn = build_make_loader_fn(
    root=ROOT, use_offline_spikes=USE_OFFLINE_SPIKES, encode_runtime=RUNTIME_ENCODE,
)

_DL_KW = dict(
    num_workers=NUM_WORKERS,
    prefetch_factor=PREFETCH,
    pin_memory=PIN_MEMORY,
    persistent_workers=PERSISTENT,
    aug_train=AUG_CFG,
    balance_train=USE_ONLINE_BAL,
    balance_bins=BAL_BINS,
    balance_smooth_eps=BAL_EPS,
)

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    """Wrapper pass-through: el runner añade dl_kwargs; aquí solo los propagamos."""
    return _raw_make_loader_fn(
        task=task, batch_size=batch_size, encoder=encoder, T=T, gain=gain, tfm=tfm, seed=seed,
        **{**_DL_KW, **dl_kwargs}
    )

print("make_loader_fn listo.")


Usando: tasks_balanced.json
Tareas y TRAIN CSV/H5 a usar:
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
make_loader_fn listo.


<a id="sec-04"></a>
## 4) Métricas y objetivo para Optuna

**Objetivo**  
Definir la **función objetivo** del HPO a partir de las métricas almacenadas:

1. Leer `continual_results.json` del *run*.  
2. Identificar **primera** y **última** tarea.  
3. Extraer:
   - `c1_mae`: MAE de la primera tarea en su propio test.  
   - `c1_after_c2_mae`: MAE de la primera **tras** aprender la última (olvido).  
   - `c2_mae`: MAE de la **última** tarea.  
4. Calcular **Olvido relativo (%)** = \((c1\_after\_c2 - c1\_mae)/c1\_mae \times 100\).

La **pérdida** a minimizar es:  
\[
\text{Objetivo}= \mathrm{MAE}_{\text{tarea final}} + \alpha \cdot \max(0, \text{olvido rel. } \%)
\]

> Ajusta `ALPHA_FORGET` para priorizar estabilidad (olvido bajo) vs. desempeño en la última tarea.  


[↑ Volver al índice](#toc)

In [4]:
# === Métricas y objetivo: usa utilidades del repo ===
from src.utils_exp import extract_metrics, safe_read_json
from src.hpo import objective_value, composite_objective
from pathlib import Path

ALPHA_FORGET = 0.5  # peso del olvido relativo en el objetivo compuesto

def _load_results(out_dir: Path) -> dict:
    """Lee outputs/<exp>/continual_results.json con manejo robusto."""
    return safe_read_json(Path(out_dir) / "continual_results.json")


<a id="sec-05"></a>
## 5) Espacios de búsqueda (por método)

**Objetivo**  
Declarar qué hiperparámetros explora Optuna:

- **ewc**  
  - `lam` \(\in [3\cdot 10^8, 2\cdot 10^9]\) (log-uniform).  
  - `fisher_batches` ∈ {200, …, 1200}.  
- **rehearsal**  
  - `buffer_size` ∈ {1000, …, 8000}.  
  - `replay_ratio` ∈ [0.05, 0.4].  
- **rehearsal+ewc**: combina ambos.  
- **as-snn**  
  - `gamma_ratio` ∈ [0.3, 0.8], `lambda_a` ∈ [1.0, 4.0], `ema` ∈ [0.70, 0.98].  
- **naive**: sin HPs.

> Puedes extender a otros métodos (p. ej., `sa-snn`, `sca-snn`, `colanet`) añadiendo su espacio de búsqueda y registrando el nombre.  


[↑ Volver al índice](#toc)

In [ ]:
def suggest_params_for_method(trial: optuna.Trial, method: str, preset: str) -> dict:
    method = method.lower()
    preset = preset.lower()

    if method == "ewc":
        lam_grid = {
            "fast":     [3e8, 5e8, 7e8, 1e9],
            "std":      [2e8, 4e8, 7e8],     # mantén la rejilla estrecha
            "accurate": [5e8, 7e8, 1e9],
        }
        fisher_grid = {
            "fast":     [300, 500],          # ↓
            "std":      [500],               # ↓
            "accurate": [800],               # si llegas a accurate, ya afinas aquí
        }
        lam = trial.suggest_categorical("lam", lam_grid.get(preset, lam_grid["std"]))
        fb  = trial.suggest_categorical("fisher_batches", fisher_grid.get(preset, fisher_grid["std"]))
        return {"lam": float(lam), "fisher_batches": int(fb)}

    elif method == "rehearsal":
        buffer_size  = trial.suggest_int("buffer_size", 1000, 8000, step=1000)
        replay_ratio = trial.suggest_float("replay_ratio", 0.05, 0.4, step=0.05)
        return {"buffer_size": buffer_size, "replay_ratio": replay_ratio}

    elif method == "rehearsal+ewc":
        # igual que arriba pero preset-aware para lam/fisher
        lam = trial.suggest_categorical("lam", lam_grid.get(preset, lam_grid["std"]))
        fisher_batches = trial.suggest_categorical("fisher_batches", fisher_grid.get(preset, fisher_grid["std"]))
        buffer_size  = trial.suggest_int("buffer_size", 1000, 8000, step=1000)
        replay_ratio = trial.suggest_float("replay_ratio", 0.05, 0.4, step=0.05)
        return {
            "buffer_size": buffer_size,
            "replay_ratio": replay_ratio,
            "lam": float(lam),
            "fisher_batches": int(fisher_batches),
        }

    elif method == "as-snn":
        gamma_ratio = trial.suggest_float("gamma_ratio", 0.3, 0.8, step=0.1)
        lambda_a    = trial.suggest_float("lambda_a", 1.0, 4.0)
        ema         = trial.suggest_float("ema", 0.70, 0.98)
        return {"gamma_ratio": gamma_ratio, "lambda_a": lambda_a, "ema": ema}

    else:
        return {}


<a id="sec-06"></a>
## 6) Estudio Optuna — un método concreto

**Objetivo**  
Optimizar **un método** específico (`METHOD_TO_OPTIMIZE`) durante `N_TRIALS`.

Flujo de cada *trial*:
1. Sugerir HPs (`suggest_*`).  
2. Construir `cfg` con esos HPs (y, si `HPO_EPOCHS` está definido, **reducir epochs** solo para HPO).  
3. Ejecutar `run_continual(...)`.  
4. Leer resultados, computar la **pérdida objetivo** y devolverla a Optuna.

**Persistencia**  
Se usa **SQLite** en `outputs/optuna/` para reanudar estudios y registrar todos los *trials* (`*_trials.csv`).

> Para métodos con *replay* se desactiva `persistent_workers` por estabilidad de DataLoader (puedes ajustar *AMP/pin_memory/workers* si lo necesitas).  

[↑ Volver al índice](#toc)

In [ ]:
# Configuración del estudio
METHOD_TO_OPTIMIZE = "ewc"   # "naive" | "ewc" | "rehearsal" | "rehearsal+ewc" | "as-snn"
N_TRIALS = 2                 # súbelo cuando estés satisfecho con tiempos/estabilidad
HPO_EPOCHS = 1               # None -> usar epochs del preset; o pon un int (ej. 3) para acelerar
REHEARSAL_NAMES = ("rehearsal", "rehearsal+ewc")

def build_cfg_with_method(base_cfg: dict, method_name: str, params: dict, hpo_epochs: int|None):
    cfg = copy.deepcopy(base_cfg)
    cfg["continual"]["method"] = method_name
    cfg["continual"]["params"] = params or {}

    # Perfil equilibrado-rápido para HPO
    cfg["optim"]["amp"] = True                 # velocidad sin mucho riesgo
    cfg["data"]["pin_memory"] = True
    cfg["data"]["persistent_workers"] = False  # mantenlo en OFF para evitar cuelgues
    # 2-4 workers es el “sweet spot” en Jupyter
    cfg["data"]["num_workers"] = min(max(2, int(cfg["data"].get("num_workers") or 2)), 4)

    if hpo_epochs is not None:
        cfg["optim"]["epochs"] = int(hpo_epochs)
    return cfg

def run_one_cfg(cfg: dict) -> tuple[Path, dict, dict]:
    out_dir, res = run_continual(
        task_list=task_list,
        make_loader_fn=make_loader_fn,
        make_model_fn=make_model_fn,
        tfm=tfm,
        cfg=cfg,
        preset_name=PRESET,
        out_root=ROOT / "outputs",
        verbose=True,
    )
    results = _load_results(out_dir) or (res if isinstance(res, dict) else {})
    return out_dir, res, results

def optuna_objective(trial: optuna.Trial):
    params = suggest_params_for_method(trial, METHOD_TO_OPTIMIZE, PRESET)
    cfg_i  = build_cfg_with_method(CFG, METHOD_TO_OPTIMIZE, params, HPO_EPOCHS)

    # === Hotfix selectivo para estabilidad con replay ===
    if METHOD_TO_OPTIMIZE.lower() in REHEARSAL_NAMES:
        cfg_i["data"]["persistent_workers"] = False
        # Opcionales si vieras inestabilidad puntual:
        # cfg_i["optim"]["amp"] = False
        # cfg_i["data"]["pin_memory"] = False
        # cfg_i["data"]["num_workers"] = min(int(cfg_i["data"].get("num_workers") or 0), 2)

    cfg_i.setdefault("naming", {})
    tag = f"{METHOD_TO_OPTIMIZE}_hpo_t{trial.number}"
    if METHOD_TO_OPTIMIZE in ("ewc", "rehearsal+ewc") and "lam" in params:
        tag += f"_lam_{params['lam']:.1e}"
    cfg_i["naming"]["tag"] = tag

    try:
        out_dir, _, results = run_one_cfg(cfg_i)
        metrics = extract_metrics(results)
        val = composite_objective(metrics, ALPHA_FORGET)
        trial.set_user_attr("out_dir", str(out_dir))
        trial.set_user_attr("metrics", metrics)
        trial.set_user_attr("method", METHOD_TO_OPTIMIZE)
        trial.set_user_attr("params", params)
        return val
    except Exception as e:
        # No tires abajo todo el estudio por un trial problemático
        trial.set_user_attr("error", repr(e))
        return float("inf")
    finally:
        # Limpieza ligera entre trials (evita fragmentación de memoria GPU)
        import gc, time
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(0.5)

# --- Persistencia Optuna en SQLite ---
OPTUNA_DIR = ROOT / "outputs" / "optuna"
OPTUNA_DIR.mkdir(parents=True, exist_ok=True)

HPO_TAG = "catgrid_v5"   # <-- CAMBIA la etiqueta (v2, v3, ...)
DB_PATH = OPTUNA_DIR / f"hpo_{METHOD_TO_OPTIMIZE}_{PRESET}_{MODEL_NAME}_{ENCODER}_T{T}_g{GAIN}_{HPO_TAG}.sqlite"
STORAGE = f"sqlite:///{DB_PATH}"
STUDY_NAME = f"HPO_{METHOD_TO_OPTIMIZE}_{PRESET}_{ENCODER}_T{T}_{HPO_TAG}"

study = optuna.create_study(
    direction="minimize",
    study_name=STUDY_NAME,
    storage=STORAGE,
    load_if_exists=False,  # <-- fuerza estudio NUEVO con ese nombre
)

study.optimize(optuna_objective, n_trials=N_TRIALS, show_progress_bar=True)

print("SQLite:", DB_PATH)
print("Best value:", study.best_value)
print("Best params:", study.best_params)
print("Best attrs:", study.best_trial.user_attrs)

# Guardar trazabilidad de intentos a CSV
df_trials = study.trials_dataframe(attrs=("number","value","state","params","user_attrs"))
df_trials.to_csv(OPTUNA_DIR / f"{DB_PATH.stem}_trials.csv", index=False)


print("Best value:", study.best_value)
print("Best params:", study.best_params)
print("Best attrs:", study.best_trial.user_attrs)

[I 2025-10-07 17:58:26,583] A new study created in RDB with name: HPO_ewc_std_rate_T16_catgrid_v4
  0%|          | 0/2 [00:00<?, ?it/s]


--- Tarea 1/2: circuito1 | preset=std | method=ewc_lam_7e+08 | B=96 T=16 AMP=True | enc=rate ---


<a id="sec-07"></a>
## 7) Estudio Optuna conjunto (elige método + HPs)

**Objetivo**  
Permitir que **cada *trial*** elija también el **método** (`naive`, `ewc`, `rehearsal`, `rehearsal+ewc`, `as-snn`) además de sus HPs. Útil cuando:

- No tienes claro qué método se adapta mejor a tu conjunto de datos.  
- Quieres una **comparativa automática** con el mismo presupuesto de cómputo.

> Activa `RUN_JOINT=True` para lanzar este estudio y eleva `N_TRIALS_JOINT` cuando el flujo sea estable.  

[↑ Volver al índice](#toc)

In [ ]:
RUN_JOINT = False   # Pon True si quieres lanzar el estudio conjunto
N_TRIALS_JOINT = 10

def optuna_objective_joint(trial: optuna.Trial):
    method = trial.suggest_categorical("method", ["naive","ewc","rehearsal","rehearsal+ewc","as_snn"])
    params = suggest_params_for_method(trial, method)
    cfg_i  = build_cfg_with_method(CFG, method, params, HPO_EPOCHS)

    # === Hotfix selectivo (solo métodos con replay) ===
    if method.lower() in REHEARSAL_NAMES:
        cfg_i["data"]["persistent_workers"] = False
        # Opcionales si hiciera falta:
        # cfg_i["optim"]["amp"] = False
        # cfg_i["data"]["pin_memory"] = False
        # cfg_i["data"]["num_workers"] = min(int(cfg_i["data"].get("num_workers") or 0), 2)

    cfg_i.setdefault("naming", {})
    tag = f"{method}_hpo_t{trial.number}"
    if method in ("ewc", "rehearsal+ewc") and "lam" in params:
        tag += f"_lam_{params['lam']:.1e}"
    cfg_i["naming"]["tag"] = tag

    try:
        out_dir, _, results = run_one_cfg(cfg_i)
        metrics = extract_metrics(results)
        val = objective_value(metrics, ALPHA_FORGET)  # tu versión conjunta
        trial.set_user_attr("out_dir", str(out_dir))
        trial.set_user_attr("metrics", metrics)
        trial.set_user_attr("method", method)
        trial.set_user_attr("params", params)
        return val
    except Exception as e:
        trial.set_user_attr("error", repr(e))
        return float("inf")
    finally:
        import gc, time
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(0.5)



if RUN_JOINT:
    study_joint = optuna.create_study(direction="minimize", study_name="HPO_joint")
    study_joint.optimize(optuna_objective_joint, n_trials=N_TRIALS_JOINT, show_progress_bar=True)
    print("Best value:", study_joint.best_value)
    print("Best params:", study_joint.best_params)
    print("Best attrs:", study_joint.best_trial.user_attrs)
else:
    print("RUN_JOINT=False — omitido.")

<a id="sec-08"></a>
## 8) Re-entrena con los mejores hiperparámetros

**Objetivo**  
Tomar `study.best_params` (y el método ganador) y **re-entrenar a pleno rendimiento**:

- Reconstruir `cfg_best` con los HPs óptimos.  
- (Opcional) Restaurar `optim.epochs` del preset si usaste `HPO_EPOCHS` reducido durante la búsqueda.  
- Ejecutar `run_continual(...)` y mostrar las **métricas finales** y la **carpeta de salida**.

> Separa la **búsqueda rápida** (menos epochs) del **entrenamiento definitivo** (epochs del preset).  

[↑ Volver al índice](#toc)

In [ ]:
# Usa el mejor del estudio por método (arriba)
BEST_PARAMS = study.best_params
BEST_METHOD = METHOD_TO_OPTIMIZE

print("Mejor método:", BEST_METHOD)
print("Mejores HPs:", BEST_PARAMS)

cfg_best = copy.deepcopy(CFG)
cfg_best["continual"]["method"] = BEST_METHOD
cfg_best["continual"]["params"] = BEST_PARAMS

# (Opcional) restablecer epochs al valor del preset si redujiste para HPO
# cfg_best["optim"]["epochs"] = load_preset(ROOT / "configs" / "presets.yaml", PRESET)["optim"]["epochs"]
# Blindaje opcional si tuviste cuelgues CUDA en re-train:
# cfg_best["optim"]["amp"] = False
# cfg_best["data"]["persistent_workers"] = False
# cfg_best["data"]["pin_memory"] = False
# cfg_best["data"]["num_workers"] = min(int(cfg_best["data"].get("num_workers") or 0), 2)

# Reponer ajustes del preset para el re-train (rápido y estable)
cfg_best["optim"]["amp"] = True
cfg_best["data"]["persistent_workers"] = CFG["data"]["persistent_workers"]
cfg_best["data"]["pin_memory"] = CFG["data"]["pin_memory"]
cfg_best["data"]["num_workers"] = CFG["data"]["num_workers"]

# (opcional) si aún va justo de memoria:
# cfg_best["optim"]["batch_size"] = min(int(cfg_best["optim"]["batch_size"]), 64)

out_dir, _, results = run_one_cfg(cfg_best)
metrics = extract_metrics(results)
print("Resultados finales (re-train):", metrics)
print("Guardado en:", out_dir)

<a id="sec-09"></a>
## 9) Resumen rápido de *runs* (tabla)

**Objetivo**  
Inspeccionar resultados de `outputs/continual_*`:

- Construir una tabla con `preset`, `method`, `encoder`, `seed` (y `lambda` si aplica),  
  junto a `c1_mae`, `c1_after_c2_mae`, `forget_rel_%`, `c2_mae`.  
- Ordenar/filtrar para comparar **manzanas con manzanas** (mismo preset/encoder).

> Exporta a CSV/Parquet para gráficas comparativas. Si la tabla sale vacía, revisa que existan `continual_results.json`.  

[↑ Volver al índice](#toc)

In [ ]:
# === Tabla mínima para HPO / inspección rápida ===
from src.utils_exp import build_runs_df
import pandas as pd

outputs_root = ROOT / "outputs"
df = build_runs_df(outputs_root)

print(f"runs en resumen: {len(df)}")
if df.empty:
    print("No hay filas (¿no existen JSONs o solo hubo 1 tarea por run?).")
else:
    # columnas clave para tuning; mantiene el nombre 'forget_rel_%' por compatibilidad
    view = df.rename(columns={"c1_forgetting_mae_rel_%": "forget_rel_%"}).loc[:, [
        "exp","preset","method","lambda","encoder","model","seed",
        "c1_mae","c1_after_c2_mae","forget_rel_%","c2_mae"
    ]]
    display(view.sort_values(["preset","method","encoder","lambda"], na_position="last", ignore_index=True))


## Apéndice — Optuna en 90 segundos

- **Study**: el proyecto de HPO (contiene todos los trials).
- **Trial**: una evaluación con un conjunto de HPs (`suggest_int`, `suggest_float`, etc.).
- **Sampler**: estrategia para elegir el siguiente punto (por defecto, TPE).
- **Pruner**: **corta** trials que pintan mal (acelera búsquedas largas).
- **Storage**: base de datos (SQLite, PostgreSQL) para **reanudar** y/o **paralelizar**.

### Reanudar búsquedas
Puedes crear el estudio con almacenamiento:
```python
study = optuna.create_study(
    direction="minimize",
    study_name="HPO_ewc",
    storage=f"sqlite:///{ROOT/'outputs'/'optuna_ewc.sqlite'}",
    load_if_exists=True,
)


In [ ]:
from src.hpo import objective_value, composite_objective
m = {"c2_mae": 0.16, "forget_rel_%": 12.5}
print(objective_value(m, key="forget_rel_%"))         # 12.5
print(objective_value(m, key="c1_forgetting_mae_rel_%"))  # 12.5 (alias OK)
print(composite_objective(m, alpha=0.5))              # 0.16 + 0.5*12.5 = 6.41
